In [ ]:
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [ ]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1045610, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [ ]:
def positioner(a,b,c,d,e):
    
    start = dt.now()
    
    riskusd = 5.00        
    symbol_info = mt5.symbol_info(a)
    point = mt5.symbol_info(a).point
    quote = a[3:]

    if quote == "USD":
        exchange = 1.00
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
    
    if quote == "JPY":
        stpips = abs(e - c)*100
        lot = round((((riskusd*exchange)/stpips)/10)/100,2)
    else:
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/stpips)/10,2)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
        
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)

    return request

In [ ]:
def livepearls(h,i,j,k,l,m,threshold,threshold2,RATE,RATE2,spy,extraj):
    
    start = dt.now()
    
    if i == "LONG":
        lasttickb=mt5.symbol_info_tick(h).ask
        if lasttickb > extraj:
            RATE2 += 1
        else:
            RATE2 = 0
        if lasttickb > j:
            RATE += 1
        else:
            RATE = 0
    elif i == "SHORT":
        lasttickb=mt5.symbol_info_tick(h).bid
        if lasttickb < extraj:
            RATE2 += 1
        else:
            RATE2 = 0
        if lasttickb < j:
            RATE += 1
        else:
            RATE = 0
    
    if RATE >= threshold:
        request = positioner(h,i,j,k,l)
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
        spy = "NO"
    elif RATE2 >= threshold2:
        request = positioner(h,i,j,k,l)
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("order_send failed, retcode={}".format(result.retcode))
            result_dict=result._asdict()
            for field in result_dict.keys():
                print("   {}={}".format(field,result_dict[field]))
                if field=="request":
                    traderequest_dict=result_dict[field]._asdict()
                    for tradereq_filed in traderequest_dict:
                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
        spy = "NO"
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    
    return RATE, RATE2, spy

In [ ]:
def levelgrab(pair):
    
    levslist = []
    
    for x in pair:
        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        y = S['close']
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        
        mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
        while mindist < SCM:
            i = 0
            h = -1
            j = 1
            for c in range(0, len(levs)):
                if j == len(levs):
                    j = -1
                if abs(levs[i]-levs[h]) < SCM:
                    levs[i] = 0
                elif abs(levs[i]-levs[j]) < SCM:
                    levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                i += 1
                h += 1
                j += 1
            levs = [v for v in levs if v != 0]
            mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
            
        levs = tuple(levs)

        levslist.append(levs)

    return levslist


In [ ]:
## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus(pair,levslist):

    start = dt.now()
    purp = []
    l = 3
    lin = np.linspace(1,l,num=l)
    ia = 0
    for x in pair:
        
        currency = x[3:]
        F = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)
        x1 = np.linspace(1, 200, num=200)
        yclose = F['close']
        if currency == "JPY":
            y1 = yclose*100
        else:
            y1 = yclose*10000
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope
        scopetheta = abs(math.degrees(math.atan(wholeslope)))
        
        if scopetheta > 5:
        
            levels = levslist[ia]

            Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 3)
            D = Drates[0]
            D['time']=pd.to_datetime(D['time'], unit='s')
            D = D.drop(columns=['tick_volume', 'spread', 'real_volume'])
            D = D.set_index('time')
            D = D.assign(MOVE=(D['close'] - D['open']))
            D = D.assign(STICK=(D['high'] - D['low']))
            D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
            D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')
            CM = np.average(abs(D['MOVE']))

            ANIMAL = []
            i = 0
            for t in lin: 
                if D.iloc[i,6] >= 0.85:
                    if D.iloc[i,6] < 0.85:
                        if D.iloc[i,6] < 0.65:
                            if D.iloc[i,6] < 0.50:
                                if D.iloc[i,6] <= 0.30:
                                    if D.iloc[i,6] <= 0.15:
                                        animal = 'Doji'
                                    else:
                                        animal = 'Frog'
                                else:
                                    animal = 'Cat'
                            else:
                                animal = 'Dog'
                        else:
                            animal = 'Horse'
                    else:
                        animal = 'Shark'
                else:
                    animal = 'Whale'
                ANIMAL.append(animal)
                i += 1
            D['ANIMAL'] = ANIMAL

            bullgauges = ['Top Saturated','Top Stacked','Top Semi-Stacked','Centered','Bottom Semi-Stacked']
            beargauges = ['Bottom Saturated','Bottom Stacked','Bottom Semi-Stacked','Centered','Top Semi-Stacked']
            GAUGE = []
            i = 0
            cndmid = (D.iloc[i,0]+D.iloc[i,3])/2
            stickmid = (D.iloc[i,1]+D.iloc[i,2])/2
            for t in lin: 
                if D.iloc[i,5] == 0:
                    midpoint = D.iloc[i,0]
                else:
                    midpoint = (cndmid - stickmid)/D.iloc[i,5]
                if midpoint >= 0.075:
                    if midpoint >= 0.175:
                        if midpoint >= 0.25:
                            gauge = 'Top Saturated'
                        else:
                            gauge = 'Top Stacked'
                    else:
                        gauge = 'Top Semi-Stacked'
                elif midpoint <= -0.075:
                    if midpoint <= -0.175:
                        if midpoint <= -0.25:
                            gauge = 'Bottom Saturated'
                        else:
                            gauge = 'Bottom Stacked'
                    else:
                        gauge = 'Bottom Semi-Stacked'
                else:
                    gauge = 'Centered'
                GAUGE.append(gauge)
                i += 1
            D['GAUGE'] = GAUGE

            TREND = []
            DTrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 168)
            DT = pd.DataFrame(DTrates)
            DT['time']=pd.to_datetime(DT['time'], unit='s')
            DT = DT.drop(columns=['tick_volume', 'spread', 'real_volume'])
            DT = DT.set_index('time')
            DT = DT.assign(MOVE=(DT['close'] - DT['open']))
            movetrend = np.average(DT['MOVE'])
            TREND.append(movetrend)
            TREND.append(movetrend)
            TREND.append(movetrend)
            D['TREND'] = TREND

            P = D.iloc[0,:]

            if wholeslope > 0:
                if P[10] > 0:
                    ditrend = 1
                elif P[10] < 0:
                    ditrend = -1
                else:
                    ditrend = 0
            elif wholeslope < 0:
                if P[10] < 0:
                    ditrend = 1
                elif P[10] > 0:
                    ditrend = -1
                else:
                    ditrend = 0
            else:
                ditrend = 0

            if ditrend == 1:

                OTESTLVL = 0
                for u in levels:
                    if abs(P[0]-u) <= CM:
                        OTESTLVL = u
                CTESTLVL = 0
                for u in levels:
                    if abs(P[3]-u) <= CM:
                        CTESTLVL = u

                if P[7] == 'Bullish':
                    if P[10] > 0:
                        dstrend = 1
                    elif P[10] < 0:
                        dstrend = -1
                    else:
                        dstrend = 0
                elif P[7] == 'Bearish':
                    if P[10] < 0:
                        dstrend = 1
                    elif P[10] > 0:
                        dstrend = -1
                    else:
                        dstrend = 0
                else:
                    dstrend = 0

                color = 'white'
                if CTESTLVL != 0:
                    if OTESTLVL == CTESTLVL:
                        color = 'gray'
                    elif OTESTLVL != CTESTLVL:
                        if dstrend == -1:
                            color = 'purple'

                if color == 'purple':
                    coltrade = 'TRADE'
                else:
                    coltrade = 0

                if coltrade == 'TRADE':

                    Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)
                    H = Hrates['close']
                    hlin = np.linspace(1,25,num=25)
                    H = savgol_filter(H, 13, 3)
                    SLOPE = []
                    i = 0
                    for t in hlin:
                        ii = i - 1
                        if i == 0:
                            SLOPE.append(0)
                        else:
                            slope = H[i] - H[ii]
                            SLOPE.append(slope)
                        i += 1
                    SLOPE2 = []
                    i = 0
                    for t in hlin:
                        ii = i - 1
                        if i == 0:
                            SLOPE2.append(0)
                        else:
                            slope2 = SLOPE[i] - SLOPE[ii]
                            SLOPE2.append(slope2)
                        i += 1
                    CONCAVE = []
                    i = 0
                    for t in hlin:
                        if SLOPE2[i] == 0:
                            CONCAVE.append('INFL')
                        elif SLOPE2[i] > 0:
                            CONCAVE.append('UP')
                        else:
                            CONCAVE.append('DOWN')
                        i += 1

                    AXCONCAVE = CONCAVE[-1]
                    if P[10] > 0:
                        if AXCONCAVE == 'UP':
                            contrade = 'TRADE'
                        else:
                            contrade = 0
                    elif P[10] < 0:
                        if AXCONCAVE == 'DOWN':
                            contrade = 'TRADE'
                        else:
                            contrade = 0
                    else:
                        contrade = 0

                    if contrade == 'TRADE':

                        if P[10] > 0:
                            position = 'LONG'
                            entry = CTESTLVL + CM
                        elif P[10] < 0:
                            position = 'SHORT'
                            entry = CTESTLVL - CM
                        else:
                            position = 0
                            entry = 0

                        cend = int(len(levels)-1)
                        if position == 'LONG':
                            clindex = levels.index(CTESTLVL)
                            tpindex = int(clindex - 1)
                            slindex = int(clindex + 1)
                            if clindex == 0:
                                stoploss = levels[slindex]
                                sldist = entry - stoploss
                                takeprofit = entry + sldist
                            elif clindex == cend:
                                takeprofit = levels[tpindex]
                                tpdist = takeprofit - entry
                                stoploss = entry - tpdist
                            else:
                                takeprofit = levels[tpindex]
                                stoploss = levels[slindex]
                        elif position == 'SHORT':
                            clindex = levels.index(CTESTLVL)
                            tpindex = int(clindex + 1)
                            slindex = int(clindex - 1)
                            if clindex == 0:
                                takeprofit = levels[tpindex]
                                tpdist = entry - takeprofit
                                stoploss = entry + tpdist
                            elif clindex == cend:
                                stoploss = levels[slindex]
                                sldist = stoploss - entry
                                takeprofit = entry - sldist
                            else:
                                takeprofit = levels[tpindex]
                                stoploss = levels[slindex]
                        else:
                            takeprofit = 0
                            stoploss = 0

                        shrink = 0.90
                        if position == 'LONG':
                            tpdist = takeprofit - entry
                            takeprofit = entry + tpdist*shrink
                        elif position == 'SHORT':
                            tpdist = entry - takeprofit
                            takeprofit = entry - tpdist*shrink

                        if position == 'LONG':
                            tpdist = takeprofit - entry
                            sldist = entry - stoploss
                            if sldist > tpdist:
                                stoploss = entry - tpdist
                        elif position == 'SHORT':
                            tpdist = entry - takeprofit
                            sldist = stoploss - entry
                            if sldist > tpdist:
                                stoploss = entry + tpdist

                        prp = (x,position,entry,takeprofit,stoploss,CM)
                        purp.append(prp)
        ia += 1

    purps = pd.DataFrame(purp, columns =['x','position','entry','takeprofit','stoploss','CM']) 
    purps['RATE'] = 0 
    purps['RATE2'] = 0
    purps['spy'] = "YES"

    purpcnt = int(purps['x'].count())
    purplin = np.linspace(1,purpcnt,num=purpcnt)
    
    EJ = []
    i = 0
    for t in purplin:
        if purps.iloc[i,1] == "LONG":
            extrajj = purps.iloc[i,2] + purps.iloc[i,5]
        elif purps.iloc[i,1] == "SHORT":
            extrajj = purps.iloc[i,2] - purps.iloc[i,5]
        EJ.append(extrajj)
        i += 1
    purps['extraj'] = EJ
    
    print(purps)
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    
    watchers = purps['spy']
    ite = 0
    while any(s == "YES" for s in watchers):
        i = 0
        for t in purplin:
            if purps.iloc[i,8] == "YES":
                Xpair = purps.iloc[i,0]
                position = purps.iloc[i,1]
                entry = purps.iloc[i,2]
                takeprofit = purps.iloc[i,3]
                stoploss = purps.iloc[i,4]
                CM = purps.iloc[i,5]
                k1 = purps.iloc[i,6]
                k2 = purps.iloc[i,7]
                k3 = purps.iloc[i,8]
                extraj = purps.iloc[i,9]
                RATE, RATE2, spy = livepearls(Xpair,position,entry,takeprofit,stoploss,CM,30,1,k1,k2,k3,extraj)
                purps.iloc[i,6] = RATE
                purps.iloc[i,7] = RATE2
                purps.iloc[i,8] = spy
            i += 1
        watchers = purps['spy']
        print(purps)
        if ite == 280:
            break
        ite += 1
        time.sleep(1)
    
    end = dt.now()
    elapsed = end - start
    elapsed = str(elapsed.total_seconds()) + '   seconds'
    print(elapsed)
    playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')

In [ ]:
print("Initializing...")

pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']
levslist = levelgrab(pair)

brutus(pair,levslist)

In [ ]:
print("Initializing...")

pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']
levslist = levelgrab(pair)

t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            brutus(pair,levslist)
    t2 = t
    time.sleep(1)